In [1]:
import mlflow

In [2]:
# Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-13-127-25-124.ap-south-1.compute.amazonaws.com:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://youtube-insights-chrome-plugin-mlflow-bucket/547399241317235994', creation_time=1750433363378, experiment_id='547399241317235994', last_update_time=1750433363378, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

c:\ProgramData\anaconda3\envs\yt-insights-ch-plugin\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [6]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Logistic Regression

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [7]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [8]:
# Step 6: Optuna objective function for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    # LogisticRegression model setup with balanced class weight
    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))

In [9]:
# Step 7: Run Optuna for Logistic Regression, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LogisticRegression"
    log_mlflow("LogisticRegression", best_model, X_train, X_test, y_train, y_test)

In [10]:
# Run the experiment for Logistic Regression
run_optuna_experiment()

[I 2025-06-20 21:22:53,995] A new study created in memory with name: no-name-ea3f0553-6db4-4c59-8315-b9ea83fa41b2
[I 2025-06-20 21:22:54,388] Trial 0 finished with value: 0.7787994081589515 and parameters: {'C': 1.4297769566595722, 'penalty': 'l1'}. Best is trial 0 with value: 0.7787994081589515.
[I 2025-06-20 21:22:54,652] Trial 1 finished with value: 0.6984781230183893 and parameters: {'C': 0.027800061445581874, 'penalty': 'l2'}. Best is trial 0 with value: 0.7787994081589515.
[I 2025-06-20 21:22:55,147] Trial 2 finished with value: 0.7673853307968717 and parameters: {'C': 1.5548180098190376, 'penalty': 'l2'}. Best is trial 0 with value: 0.7787994081589515.
[I 2025-06-20 21:22:55,279] Trial 3 finished with value: 0.3333333333333333 and parameters: {'C': 0.0030141151682067206, 'penalty': 'l1'}. Best is trial 0 with value: 0.7787994081589515.
[I 2025-06-20 21:22:55,517] Trial 4 finished with value: 0.7454026632847178 and parameters: {'C': 0.11501470789217681, 'penalty': 'l1'}. Best is 